In [28]:
from datasets import Dataset, load_from_disk
import evaluate



dataset_name = '2024_gpt35_span'

seqeval = evaluate.load("seqeval")
dataset = load_from_disk(f'../data/{dataset_name}')
dataset

DatasetDict({
    train: Dataset({
        features: ['idx', 'tokens', 'ner_tags'],
        num_rows: 18185
    })
    dev: Dataset({
        features: ['idx', 'tokens', 'ner_tags'],
        num_rows: 965
    })
})

In [29]:
import re


label_list = dataset["train"].features[f"ner_tags"].feature.names

true_labels = [[label_list[a] for a in entry if a not in (-100, 3)] for entry in dataset['dev']['ner_tags']]

predicted = []
for entry in dataset['dev']['ner_tags']:
    reduced = ''.join([str(a) for a in entry])
    reduced = re.sub(r'3{3}', '3', reduced)
    reduced = [int(a) for a in reduced]
    
    result = []
    next_token = 0
    for a in reduced:
        if a == 3:
            if next_token == 0:
                next_token = 1
            else:
                next_token = 0
                
            continue

        result.append(next_token)
        
        if next_token == 1:
            next_token = 2

    predicted.append(result)
    
predicted = [[label_list[a] for a in entry] for entry in predicted]
seqeval.compute(predictions=predicted, references=true_labels)

{'ADE': {'precision': 0.08365758754863813,
  'recall': 0.524390243902439,
  'f1': 0.1442953020134228,
  'number': 82},
 'overall_precision': 0.08365758754863813,
 'overall_recall': 0.524390243902439,
 'overall_f1': 0.1442953020134228,
 'overall_accuracy': 0.9373234557373554}